CFLP

Capacitated facility location problem
---

# Formulation

Open a **subset** of facilities & **connect** customers to _one facility each_ at a _minimal cost_

* $f_j$ fixed cost associated with opening the facility j
* $c_{ij}$ variable cost of connecting customer i to facility j
* $q_{i}$ demand associated to customer i
* $V_j$ capacity of facility j

$min_x$ 

$$\sum_{j \in J} f_{j} z_j + \sum_{i \in I}\sum_{j \in J} c_{ij} x_{ij}$$

$s.t.$
$$\tag{1} \sum_{j \in J} x_{ij} = 1, \forall i \in I$$
$$\tag{2} \sum_{i \in I} q_{i} x_{ij} \le V_j z_j, \forall j \in J$$
$$x_{ij}, z_{j} \in {0,1}$$


* $z_j$ decision variable associated with openning facility j
* $x_{ij}$ decision variable associated with connecting customer i to facility j

* (1) ensure that each customer is associated to a unique facility
* (2) ensure that each customer is associated with an open facility and that the capacity constraint is respected

# Solve with Cplex

In [3]:
%%file "../pkg/cflp_cplex.py"

import numpy as np              # mathematic tools library
import networkx as nx           # network representation library
from pkg.cplex_solve import cplex_solve

def cflp_cplex(I,J,
            c,f,
            q,V,
            relaxation=False,path=None):
    """
    I,J number of customer and facilities
    v,f cost matrices for variables cost and fixed cost
    q,V demand of customers and capacity of facilities
    """
    #####################################################################
    # Decision variables
    def X(i,j):
        return "X_" + str(i) + "_" + str(j)

    def Z(j):
        return "Z_" + str(j)
    
    I = range(I); J = range(J);
    #####################################################################
    # Objective function
    
    Xs = {
        "name" : [X(i,j) for i in I for j in J],
        "coef" : [c[i][j] for i in I for j in J],
        "type" : ["C" if relaxation else "I" for i in I for j in J],
        "ub"   : [1 for i in I for j in J],
        "lb"   : [0 for i in I for j in J],
    }
    
    Zs = {
        "name" : [Z(j) for j in J],
        "coef" : [f[j] for j in J],
        "type" : ["C" if relaxation else "I" for j in J],
        "ub"   : [1 for j in J],
        "lb"   : [0 for j in J],
    }
    
    variables = [Xs, Zs]
    #####################################################################
    # Constraints
    
    c1 = {
        "lin_expr": [[[X(i,j) for j in J], [1 for j in J]]
                     for i in I],
        "senses"  : ["E" for i in I],
        "rhs"     : [0 for i in I]
    }
    
    c1 = {
        "lin_expr": [[[X(i,j) for i in range(J)]+[Z(j)],
                       [q[i] for i in range(J)]+[-V[j]]] 
        for j in range(J)],
        "senses"  : ["L" for j in range(J)],
        "rhs"     : [0 for j in range(J)]
    }
    
    constraints = [c1, c2]
    #####################################################################
    # Solving
    prob = cplex_solve(variables,constraints,
                       minimize=True, path=path, verbose=verbose)

    #####################################################################
    # Extract solution
    solution = prob.solution.get_values()
    I = len(I); J = len(J);
    
    X = np.reshape(solution[0:I*J],(I,J))
    Z = solution[I*J:I*J+J]

    return prob, X, Z

Overwriting ../pkg/cflp_cplex.py
